In [73]:
import numpy as np
from dppy.finite_dpps import FiniteDPP

r, N = 4, 10 #r number of features and N number of videos

#L-Ensemble of rank r that is not a projection DPP
Phi = np.random.randn(r, N)
DPP_L = FiniteDPP('likelihood', **{'L_gram_factor': Phi})

#Projection DPP with the corresponding previous eigenvectors
L = np.matmul(np.transpose(Phi), Phi)
e_vals, e_vecs = np.linalg.eigh(L)
proj_DPP = FiniteDPP('correlation', projection=True,
                    **{'K_eig_dec': (np.concatenate((np.zeros(N-r),np.ones(r))), e_vecs)})

L_dual = Phi Phi.T was computed: Phi (dxN) with d<N


In [50]:
for _ in range(10):
    print(DPP_L.sample_exact())

[5, 9, 6]
[0, 9, 1, 4]
[9, 4]
[9, 1, 3, 6]
[6, 1, 0]
[5, 6, 4]
[8, 3, 0]
[5, 0, 2]
[1, 9, 4]
[4, 9, 1]


In [81]:
for _ in range(10):
    print(proj_DPP.sample_exact())

[5, 6, 0, 1]
[7, 2, 5, 1]
[0, 6, 7, 3]
[3, 4, 7, 0]
[8, 2, 3, 7]
[1, 7, 3, 0]
[2, 1, 0, 6]
[6, 0, 3, 4]
[1, 0, 2, 6]
[1, 6, 2, 0]
